In [6]:
import numpy as np
import pulp as pl
from numpy.typing import NDArray
from typing import Literal
from scipy.linalg import lu

In [7]:
def to_standard_equality_form(
    type: Literal["min", "max"], 
    A: NDArray[np.float64], 
    b: NDArray[np.float64], 
    c: NDArray[np.float64], 
    restriction_types: list[str], 
    non_negative_variables: list[bool] = []
) -> tuple[NDArray[np.float64], NDArray[np.float64], NDArray[np.float64]]:
    """
    Converts a linear programming problem to an equivalent problem in standard equality form.
    Parameters:
    A : 2D array
        Coefficient matrix of the constraints.
    b : 1D array
        Right-hand side vector of the constraints.
    c : 1D array
        Coefficient vector of the objective function.
    restriction_types : list of str
        List indicating the type of each constraint ('<=', '>=', '=').
    non_negative_variables : list of bool, optional
        List indicating whether each variable is non-negative. If empty, all variables are assumed to be non-negative.
    """
    # Number of constraints and number of variables
    m, n = A.shape

    # Convert minimization to maximization
    if type == "min":
        c = -c
        
    # Impose non-negativity on variables
    count_free = 0
    for (i, non_negative) in enumerate(non_negative_variables):
        if not non_negative:
            curr_i = i + count_free
            # Adjusting the objective function
            c = np.insert(c, curr_i+1, -c[curr_i])
            
            # Adjusting the constraint matrix
            A = np.insert(A, curr_i+1, -A[:, curr_i], axis=1)
            n += 1
            count_free += 1
            
    # Convert inequalities to equalities
    for (i, restriction) in enumerate(restriction_types):
        if restriction == "<=":
            # Add slack variable
            slack = np.zeros((m, 1))
            slack[i, 0] = 1
            A = np.hstack((A, slack))
            c = np.hstack((c, [0]))
            n += 1
        elif restriction == ">=":
            # Add surplus variable
            surplus = np.zeros((m, 1))
            surplus[i, 0] = -1
            A = np.hstack((A, surplus))
            c = np.hstack((c, [0]))
            n += 1
        
    return A, b, c

In [ ]:
def to_canonical_form(
    B: list[int],
    N: list[int],
    A: NDArray[np.float64], 
    b: NDArray[np.float64], 
    c: NDArray[np.float64], 
    z: np.float64
) -> tuple[NDArray[np.float64], NDArray[np.float64], NDArray[np.float64]]:
    """
    Converts a linear programming problem to an equivalent problem in canonical form.
    Parameters:
    B: list of int
        List of indices representing the basic variables.
    N: list of int
        List of indices representing the non-basic variables.
    A : 2D array
        Coefficient matrix of the constraints.
    b : 1D array
        Right-hand side vector of the constraints.
    c : 1D array
        Coefficient vector of the objective function.
    restriction_types : list of str
        List indicating the type of each constraint ('<=', '>=', '=').
    non_negative_variables : list of bool, optional
        List indicating whether each variable is non-negative. If empty, all variables are assumed to be non-negative.
    """
    m, n = A.shape

    # Extract the submatrix corresponding to the chosen columns
    Ab = A[:, B]
    
    Ab_inv = np.linalg.inv(Ab)
    A_canon = Ab_inv @ A
    b_canon = Ab_inv @ b # b_canon is the solution for the basic variables
    
    if np.all(b_canon >= 0):  # Check if the solution is feasible (non-negative)
        # Transform to canonical form
        y = np.linalg.solve(Ab.T, c[list(B)]) # y = (A_b ^T)^-1 * c_b
        z_canon = z + y @ b
        c_canon = c - y @ A
        x_canon = np.zeros(n)
        x_canon[list(B)] = b_canon
        return A_canon, x_canon, b_canon, c_canon, z_canon

    raise ValueError("No basic feasible solution found; the problem may be infeasible.")

In [ ]:
def phase_two_simplex(
    type: Literal["min", "max"],
    B: list[int], 
    A: NDArray[np.float64], 
    b: NDArray[np.float64], 
    c: NDArray[np.float64],
    z: np.float64, 
    restriction_types: list[str], 
    non_negative_variables: list[bool] = []
):
    '''
    Solves a linear programming problem using the Simplex method.
    Parameters:
    type : str
        Type of the problem, either "min" for minimization or "max" for maximization.
    B: list of int, optional
        List of indices representing the basic variables.
    A : 2D array
        Coefficient matrix of the constraints.
    b : 1D array
        Right-hand side vector of the constraints.
    c : 1D array
        Coefficient vector of the objective function.
    z : 1D array
        Constant term in the objective function.
    restriction_types : list of str
        List indicating the type of each constraint ('<=', '>=', '=').
    non_negative_variables : list of bool, optional
        List indicating whether each variable is non-negative. If empty, all variables are assumed to be non-negative.
    Returns:
        A : 2D array
    '''
    
    m, n = A.shape
    N = [j for j in range(n) if j not in B]
    
    while True:        
        # Step 1: Convert to canonical form
        A, x, b, c, z = to_canonical_form(B, N, A, b, c, z)
        
        # Step 2: Reached optimal solution
        if c[N].max() <= 0:
            opt_value = z + c @ x
            return x, opt_value
        
        # Step 3: Select k such that c[k] > 0
        k = 0
        for i in N:
            if c[i] > 0:
                k = i
                break
        
        # Step 4: If A_k <= then stop (the problem is unbounded)
        if A[:, k].max() <= 0:
            raise ValueError("The problem is unbounded.")
        
        # Step 5: Determine the index r and the value t for the next iteration of the simplex method.
        ratios = []
        for i in range(len(b)):
            if A[i, k] > 0:
                ratios.append((b[i] / A[i, k], i))
        t, r = min(ratios, key=lambda x: x[0])
        
        # Step 6: Let l be the rth basis element
        l = B[r]
        
        # Step 7: Set B = B \ {l} ∪ {k} and N = N \ {k} ∪ {l}
        B[r] = k
        
        N.remove(k)
        N.append(l)

        # 2.6 Atualiza A, b, c, z (pivot)
        Ab = A[:, B]
        Ab_inv = np.linalg.inv(Ab)
        A = Ab_inv @ A
        b = Ab_inv @ b
        y = np.linalg.solve(Ab.T, c[B])
        z = z + y @ b
        c = c - y @ A


In [32]:
def phase_one_simplex(A: NDArray[np.float64], b: NDArray[np.float64]):
    """
    Simplex Phase I: finds an initial feasible basis for Ax = b, x >= 0.

    Returns:
        feasible_basis : list[int] or None if the LP is infeasible.
    """
    m, n = A.shape

    # Add artificial variables to each constraint
    I_art = np.eye(m)
    A1 = np.hstack((A, I_art))
    c1 = np.zeros(n + m)
    c1[n:] = 1.0  # Minimize the sum of artificial variables

    # Initial basis: artificial variables
    B = list(range(n, n + m))
    N = [j for j in range(n)]

    max_iter = 1000
    for _ in range(max_iter):
        # Step 1: Compute inverse of the current basis matrix
        A_B = A1[:, B]
        try:
            A_B_inv = np.linalg.inv(A_B)
        except np.linalg.LinAlgError:
            return None

        # Basic feasible solution
        x_B = A_B_inv @ b

        # Step 2: Compute reduced costs
        c_B = c1[B]
        c_N = c1[N]
        y = c_B @ A_B_inv
        reduced_costs = c_N - y @ A1[:, N]

        # Step 3: Check optimality (all reduced costs ≥ 0 → optimum)
        if np.all(reduced_costs >= -1e-10):
            # Verify if artificial variables remaining in basis are zero
            for i, j in enumerate(B):
                if j >= n and abs(x_B[i]) > 1e-10:
                    return None  # infeasible
            break

        # Step 4: Choose entering variable (most negative reduced cost)
        enter_idx = np.argmin(reduced_costs)
        entering = N[enter_idx]

        # Step 5: Compute direction vector
        direction = A_B_inv @ A1[:, entering]
        if np.all(direction <= 1e-10):
            return None  # unbounded (should not occur in Phase I)

        # Step 6: Minimum ratio test
        ratios = [(x_B[i] / direction[i], i) for i in range(m) if direction[i] > 1e-10]
        if not ratios:
            return None
        _, leave_pos = min(ratios, key=lambda x: (x[0], x[1]))
        leaving = B[leave_pos]

        # Step 7: Update basis and non-basis sets
        B[leave_pos] = entering
        N[enter_idx] = leaving
        B.sort()
        N.sort()

    # Step 8: Extract feasible basis for the original problem
    feasible_B = [j for j in B if j < n]

    # Step 9: Complete the basis if necessary
    if len(feasible_B) < m:
        for j in range(n):
            if j not in feasible_B:
                trial_basis = feasible_B + [j]
                if np.linalg.matrix_rank(A[:, trial_basis]) == len(trial_basis):
                    feasible_B = trial_basis
                    if len(feasible_B) == m:
                        break

    return feasible_B if len(feasible_B) == m else None


In [44]:
def solve_lp(
    type: Literal["min", "max"],
    A: NDArray[np.float64], 
    b: NDArray[np.float64], 
    c: NDArray[np.float64],
    z: np.float64, 
    restriction_types: list[str], 
    non_negative_variables: list[bool] = []
) -> tuple[NDArray[np.float64], float]:
    '''
    Solves a linear programming problem using the Simplex method.
    Parameters:
    type : str
        Type of the problem, either "min" for minimization or "max" for maximization.
    A : 2D array
        Coefficient matrix of the constraints.
    b : 1D array
        Right-hand side vector of the constraints.
    c : 1D array
        Coefficient vector of the objective function.
    z : 1D array
        Constant term in the objective function.
    restriction_types : list of str
        List indicating the type of each constraint ('<=', '>=', '=').
    non_negative_variables : list of bool, optional
        List indicating whether each variable is non-negative. If empty, all variables are assumed to be non-negative.
    Returns:
        A tuple containing:
        - x : 1D array
            Solution vector.
        - opt_value : float
            Optimal value of the objective function.
    '''
    try: 
        # Step 0: Convert to standard equality form first
        A, b, c = to_standard_equality_form(type, A, b, c, restriction_types, non_negative_variables)
        
        B = phase_one_simplex(A, b)
        
        if B is None:
            ValueError("The problem is infeasible.")
        
        # Step 1: Phase 2 - solve the original problem starting from the basic feasible solution found in Phase 1
        return phase_two_simplex(type, B, A, b, c, z, restriction_types, non_negative_variables)
    
    except ValueError as e:
        print(e)
        return None, None

## 1.2.1 - Nutrição


In [12]:
import pulp as pl

def solve_nutrition_problem():
    # Criação do problema
    lp = pl.LpProblem("Nutrition_Optimization", pl.LpMinimize)

    # Variáveis de decisão (servings de cada alimento)
    x1 = pl.LpVariable("Raw_carrots", lowBound=0)
    x2 = pl.LpVariable("Baked_potatoes", lowBound=0)
    x3 = pl.LpVariable("Wheat_bread", lowBound=0)
    x4 = pl.LpVariable("Cheddar_cheese", lowBound=0)
    x5 = pl.LpVariable("Peanut_butter", lowBound=0)

    # Função objetivo (minimizar custo total)
    lp += 0.14*x1 + 0.12*x2 + 0.20*x3 + 0.75*x4 + 0.15*x5, "Total_Cost"

    # Restrições nutricionais
    lp += 23*x1 + 171*x2 + 65*x3 + 112*x4 + 188*x5 >= 2000, "Calories"
    lp += 0.1*x1 + 0.2*x2 + 0*x3 + 9.3*x4 + 16*x5 >= 50, "Fat"
    lp += 0.6*x1 + 3.7*x2 + 2.2*x3 + 7*x4 + 7.7*x5 >= 100, "Protein"
    lp += 6*x1 + 30*x2 + 13*x3 + 0*x4 + 2*x5 >= 250, "Carbohydrate"

    # Resolver
    lp.solve(pl.PULP_CBC_CMD(msg=0))

    # Solução
    solution = {var.name: var.varValue for var in [x1, x2, x3, x4, x5]}
    objective_value = pl.value(lp.objective)
    
    return solution, objective_value

solve_nutrition_problem()

({'Raw_carrots': 0.0,
  'Baked_potatoes': 7.7146691,
  'Wheat_bread': 0.0,
  'Cheddar_cheese': 0.0,
  'Peanut_butter': 9.2799642},
 2.3177549219999998)

In [13]:
c = np.array([0.14, 0.12, 0.20, 0.75, 0.15])

A = np.array([
    [23,   171,  65,  112,  188],   # Calories
    [0.1,  0.2,  0,   9.3,  16],    # Fat
    [0.6,  3.7,  2.2, 7,    7.7],   # Protein
    [6,    30,   13,  0,    2]      # Carbohydrate
])

b = np.array([2000, 50, 100, 250])

signs = np.array([">=", ">=", ">=", ">="])  # Todas são "≥"

# Vetor de booleanos para não-negatividade (todas as variáveis são ≥ 0)
nonneg = np.array([True, True, True, True, True])

z = 0

x, z = solve_lp("min", A, b, c, z, signs, nonneg)
x, z

(array([   0.        ,    7.71466905,    0.        ,    0.        ,
           9.27996422, 1063.84168157,  100.02236136,    0.        ,
           0.        ]),
 np.float64(-2.317754919499106))

## 1.2.2 - MUCOW


In [34]:
def solve_mucow_problem():
    # Criação do problema (MAXIMIZAR lucro)
    lp = pl.LpProblem("MUCOW_Production", pl.LpMaximize)

    # Variáveis de decisão (quantidade de cada produto)
    x1 = pl.LpVariable("Skimmed_milk", lowBound=0)
    x2 = pl.LpVariable("Two_percent_milk", lowBound=0)
    x3 = pl.LpVariable("Whole_milk", lowBound=0)
    x4 = pl.LpVariable("Table_cream", lowBound=0)
    x5 = pl.LpVariable("Whipping_cream", lowBound=0)

    # Função objetivo (maximizar lucro total)
    lp += 0.1*x1 + 0.15*x2 + 0.2*x3 + 0.5*x4 + 1.2*x5, "Total_Profit"

    # Restrições
    # 1. Volume total disponível
    lp += 2*x1 + 2*x2 + 2*x3 + 0.6*x4 + 0.3*x5 <= 750, "Total_Volume"
    
    # 2. Balanço de gordura total
    lp += 0*x1 + 0.04*x2 + 0.08*x3 + 0.09*x4 + 0.135*x5 == 27.75, "Fat_Balance"
    
    # 3. Composição exata de gordura para cada produto
    lp += 0*x1 == 0, "Skimmed_milk_fat"           # 0% gordura
    lp += 0.04*x2 - 0.02*(2*x2) == 0, "Two_percent_fat"    # 2% gordura
    lp += 0.08*x3 - 0.04*(2*x3) == 0, "Whole_milk_fat"     # 4% gordura
    lp += 0.09*x4 - 0.15*(0.6*x4) == 0, "Table_cream_fat"  # 15% gordura
    lp += 0.135*x5 - 0.45*(0.3*x5) == 0, "Whipping_cream_fat" # 45% gordura

    # Resolver
    lp.solve(pl.PULP_CBC_CMD(msg=0))

    # Solução
    print("Status:", pl.LpStatus[lp.status])
    solution = {var.name: var.varValue for var in [x1, x2, x3, x4, x5]}
    objective_value = pl.value(lp.objective)
    
    return solution, objective_value

solve_mucow_problem()

Status: Optimal


({'Skimmed_milk': 344.16667,
  'Two_percent_milk': 0.0,
  'Whole_milk': 0.0,
  'Table_cream': 0.0,
  'Whipping_cream': 205.55556},
 281.083339)

In [ ]:
A2 = np.array([
    [1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 1, 1, 1, 1, 1],
    [0.037, 0, 0, 0, 0, 0.049, 0, 0, 0, 0],
    [0, 0.017, 0, 0, 0, 0, 0.029, 0, 0, 0],
    [0, 0, -0.003, 0, 0, 0, 0, 0.009, 0, 0],
    [0, 0, 0, -0.113, 0, 0, 0, 0, -0.101, 0],
    [0, 0, 0, 0, -0.413, 0, 0, 0, 0, -0.401]
])

b2 = np.array([500, 250, 0, 0, 0, 0, 0])
c2 = np.array([
    0.1/2,    # x_HS
    0.15/2,   # x_HT
    0.2/2,    # x_HW
    0.5/0.6,  # x_HC
    1.2/0.3,  # x_HWhip
    0.1/2,    # x_JS
    0.15/2,   # x_JT
    0.2/2,    # x_JW
    0.5/0.6,  # x_JC
    1.2/0.3   # x_JWhip
])
sinais2 = ['<=', '<=', '=', '=', '=', '=', '=']

x, z = solve_lp("max", A2, b2, c2, 0, sinais2)
x, z

chehou aqui
No basic feasible solution found; the problem may be infeasible.


(None, None)

## 1.2.3 - Salários


In [36]:
def solve_co_tech_salaries_min_total():
    # Criação do problema
    lp = pl.LpProblem("CO_Tech_Salaries_Min_Total", pl.LpMinimize)

    # Variáveis de decisão (salários de cada funcionário)
    Tom = pl.LpVariable("Tom", lowBound=0)
    Peter = pl.LpVariable("Peter", lowBound=0)
    Nina = pl.LpVariable("Nina", lowBound=0)
    Samir = pl.LpVariable("Samir", lowBound=0)
    Gary = pl.LpVariable("Gary", lowBound=0)
    Linda = pl.LpVariable("Linda", lowBound=0)
    Bob = pl.LpVariable("Bob", lowBound=0)

    # Função objetivo (minimizar custo total de salários)
    lp += Tom + Peter + Nina + Samir + Gary + Linda + Bob, "Total_Salary_Cost"

    # Restrições
    # Tom wants at least $20,000
    lp += Tom >= 20000, "Tom_min_salary"
    
    # Peter, Nina, Samir each want at least $5000 more than Tom
    lp += Peter - Tom >= 5000, "Peter_vs_Tom"
    lp += Nina - Tom >= 5000, "Nina_vs_Tom"
    lp += Samir - Tom >= 5000, "Samir_vs_Tom"
    
    # Gary wants at least as high as Tom + Peter
    lp += Gary >= Tom + Peter, "Gary_vs_Tom_Peter"
    
    # Linda wants her salary to be $200 more than Gary
    lp += Linda == Gary + 200, "Linda_vs_Gary"
    
    # Combined salary of Nina and Samir ≥ 2*(Tom + Peter)
    lp += Nina + Samir >= 2 * (Tom + Peter), "Nina_Samir_vs_Tom_Peter"
    
    # Bob's salary ≥ Peter and ≥ Samir
    lp += Bob >= Peter, "Bob_vs_Peter"
    lp += Bob >= Samir, "Bob_vs_Samir"
    
    # Combined salary of Bob and Peter ≥ $60,000
    lp += Bob + Peter >= 60000, "Bob_Peter_min_combined"
    
    # Linda ≤ combined salary of Bob and Tom
    lp += Linda <= Bob + Tom, "Linda_vs_Bob_Tom"

    # Resolver
    lp.solve(pl.PULP_CBC_CMD(msg=0))

    # Solução
    print("Status:", pl.LpStatus[lp.status])
    employees = ["Tom", "Peter", "Nina", "Samir", "Gary", "Linda", "Bob"]
    variables = [Tom, Peter, Nina, Samir, Gary, Linda, Bob]
    
    solution = {}
    total_salary = 0
    print("\nOptimal Salaries:")
    for emp, var in zip(employees, variables):
        salary = var.varValue
        solution[emp] = salary
        total_salary += salary
        print(f"  {emp}: ${salary:,.2f}")
    
    print(f"\nTotal Salary Cost: ${total_salary:,.2f}")
    objective_value = pl.value(lp.objective)
    
    return solution, objective_value

solve_co_tech_salaries_min_total()

Status: Optimal

Optimal Salaries:
  Tom: $20,000.00
  Peter: $25,000.00
  Nina: $65,000.00
  Samir: $25,000.00
  Gary: $45,000.00
  Linda: $45,200.00
  Bob: $35,000.00

Total Salary Cost: $260,200.00


({'Tom': 20000.0,
  'Peter': 25000.0,
  'Nina': 65000.0,
  'Samir': 25000.0,
  'Gary': 45000.0,
  'Linda': 45200.0,
  'Bob': 35000.0},
 260200.0)

In [35]:
# Variáveis: [Tom, Peter, Nina, Samir, Gary, Linda, Bob]
c = np.array([1, 1, 1, 1, 1, 1, 1])  # Minimizar soma total dos salários

# Matriz A de restrições
A = np.array([
    # Tom wants at least $20,000
    [1, 0, 0, 0, 0, 0, 0],
    
    # Peter, Nina, Samir each want at least $5000 more than Tom
    [-1, 1, 0, 0, 0, 0, 0],
    [-1, 0, 1, 0, 0, 0, 0],
    [-1, 0, 0, 1, 0, 0, 0],
    
    # Gary wants at least as high as Tom + Peter
    [-1, -1, 0, 0, 1, 0, 0],
    
    # Linda wants $200 more than Gary
    [0, 0, 0, 0, -1, 1, 0],
    
    # Nina + Samir ≥ 2*(Tom + Peter)
    [-2, -2, 1, 1, 0, 0, 0],
    
    # Bob ≥ Peter and Bob ≥ Samir
    [0, -1, 0, 0, 0, 0, 1],
    [0, 0, 0, -1, 0, 0, 1],
    
    # Bob + Peter ≥ $60,000
    [0, 1, 0, 0, 0, 0, 1],
    
    # Linda ≤ Bob + Tom
    [-1, 0, 0, 0, 0, 1, -1]
])

b = np.array([20000, 5000, 5000, 5000, 0, 200, 0, 0, 0, 60000, 0])

signs = np.array([">=", ">=", ">=", ">=", ">=", "=", ">=", ">=", ">=", ">=", "<="])

# Todas as variáveis são não-negativas (salários)
nonneg = np.array([True, True, True, True, True, True, True])

z = 0

x, z = solve_lp("min", A, b, c, z, signs, nonneg)
x, z

(array([20000., 25000., 55000., 35000., 45000., 45200., 35000.,     0.,
            0., 30000., 10000.,     0.,     0., 10000.,     0.,     0.,
         9800.]),
 np.float64(-260200.0))

## 1.2.5


In [38]:
def solve_crud_chemical_problem():
    # Criação do problema
    lp = pl.LpProblem("CRUD_Chemical_Plant", pl.LpMinimize)

    # Variáveis de decisão (litros enviados a cada hora)
    x10 = pl.LpVariable("Send_10am", lowBound=0)
    x11 = pl.LpVariable("Send_11am", lowBound=0)
    x12 = pl.LpVariable("Send_12pm", lowBound=0)
    x13 = pl.LpVariable("Send_1pm", lowBound=0)
    x14 = pl.LpVariable("Send_2pm", lowBound=0)
    x15 = pl.LpVariable("Send_3pm", lowBound=0)

    # Função objetivo (minimizar custo total de reciclagem)
    lp += 30*x10 + 40*x11 + 35*x12 + 45*x13 + 38*x14 + 50*x15, "Total_Recycling_Cost"

    # Produção por hora
    production = [300, 240, 600, 200, 300, 900]  # 9-10, 10-11, 11-12, 12-1, 1-2, 2-3
    
    # Preços de reciclagem
    prices = [30, 40, 35, 45, 38, 50]  # 10am, 11am, 12pm, 1pm, 2pm, 3pm

    # Restrições de estoque máximo (não pode exceder 1000 litros a qualquer momento)
    # Estoque às 11am: (300 - x10) + 240 ≤ 1000
    lp += 300 - x10 + 240 <= 1000, "Stock_11am"
    
    # Estoque às 12pm: (300 - x10) + (240 - x11) + 600 ≤ 1000
    lp += 300 - x10 + 240 - x11 + 600 <= 1000, "Stock_12pm"
    
    # Estoque às 1pm: (300 - x10) + (240 - x11) + (600 - x12) + 200 ≤ 1000
    lp += 300 - x10 + 240 - x11 + 600 - x12 + 200 <= 1000, "Stock_1pm"
    
    # Estoque às 2pm: (300 - x10) + ... + (200 - x13) + 300 ≤ 1000
    lp += 300 - x10 + 240 - x11 + 600 - x12 + 200 - x13 + 300 <= 1000, "Stock_2pm"
    
    # Estoque às 3pm: (300 - x10) + ... + (300 - x14) + 900 ≤ 1000
    lp += 300 - x10 + 240 - x11 + 600 - x12 + 200 - x13 + 300 - x14 + 900 <= 1000, "Stock_3pm"

    # Restrição de não deixar estoque overnight (todo chemical X deve ser enviado)
    total_production = sum(production)
    lp += x10 + x11 + x12 + x13 + x14 + x15 == total_production, "No_Overnight_Stock"

    # Restrições de capacidade máxima de envio (não pode enviar mais do que o disponível)
    lp += x10 <= 300, "Max_Send_10am"
    lp += x10 + x11 <= 300 + 240, "Max_Send_11am"
    lp += x10 + x11 + x12 <= 300 + 240 + 600, "Max_Send_12pm"
    lp += x10 + x11 + x12 + x13 <= 300 + 240 + 600 + 200, "Max_Send_1pm"
    lp += x10 + x11 + x12 + x13 + x14 <= 300 + 240 + 600 + 200 + 300, "Max_Send_2pm"
    lp += x10 + x11 + x12 + x13 + x14 + x15 <= total_production, "Max_Send_3pm"

    # Resolver
    lp.solve(pl.PULP_CBC_CMD(msg=0))

    # Solução
    print("Status:", pl.LpStatus[lp.status])
    
    times = ["10am", "11am", "12pm", "1pm", "2pm", "3pm"]
    variables = [x10, x11, x12, x13, x14, x15]
    
    solution = {}
    total_cost = 0
    print("\nOptimal Recycling Schedule:")
    for time, var, price in zip(times, variables, prices):
        liters = var.varValue
        cost = liters * price
        solution[time] = liters
        total_cost += cost
        print(f"  {time}: {liters:.1f} liters (${cost:,.2f})")
    
    print(f"\nTotal Recycling Cost: ${total_cost:,.2f}")
    
    # Verificar estoques
    print("\nInventory Levels:")
    inventory = 0
    for i in range(6):
        inventory += production[i]
        if i < 5:
            inventory -= variables[i].varValue
        stock_cost = f" (${inventory * prices[i]:.2f} if recycled now)" if i > 0 else ""
        print(f"  {9+i}-{10+i}am: {inventory:.1f} liters{stock_cost}")
    
    objective_value = pl.value(lp.objective)
    
    return solution, objective_value

solve_crud_chemical_problem()

Status: Optimal

Optimal Recycling Schedule:
  10am: 300.0 liters ($9,000.00)
  11am: 0.0 liters ($0.00)
  12pm: 840.0 liters ($29,400.00)
  1pm: 0.0 liters ($0.00)
  2pm: 500.0 liters ($19,000.00)
  3pm: 900.0 liters ($45,000.00)

Total Recycling Cost: $102,400.00

Inventory Levels:
  9-10am: 0.0 liters
  10-11am: 240.0 liters ($9600.00 if recycled now)
  11-12am: 0.0 liters ($0.00 if recycled now)
  12-13am: 200.0 liters ($9000.00 if recycled now)
  13-14am: 0.0 liters ($0.00 if recycled now)
  14-15am: 900.0 liters ($45000.00 if recycled now)


({'10am': 300.0,
  '11am': 0.0,
  '12pm': 840.0,
  '1pm': 0.0,
  '2pm': 500.0,
  '3pm': 900.0},
 102400.0)

In [37]:
# Variáveis: [x1, x2, x3, x4, x5, x6] = litros enviados às 10am, 11am, 12pm, 1pm, 2pm, 3pm
c = np.array([30, 40, 35, 45, 38, 50])  # Custos de reciclagem por litro

# Matriz A de restrições (estoque máximo e não-negatividade implícita)
A = np.array([
    # Restrição de estoque máximo às 10am: 300 - x1 ≤ 1000 (sempre satisfeita)
    # Restrição de estoque máximo às 11am: (300 - x1) + 240 - x2 ≤ 1000
    [-1, -1, 0, 0, 0, 0],
    
    # Restrição de estoque máximo às 12pm: (300 - x1) + (240 - x2) + 600 - x3 ≤ 1000
    [-1, -1, -1, 0, 0, 0],
    
    # Restrição de estoque máximo às 1pm: (300 - x1) + ... + 200 - x4 ≤ 1000
    [-1, -1, -1, -1, 0, 0],
    
    # Restrição de estoque máximo às 2pm: (300 - x1) + ... + 300 - x5 ≤ 1000
    [-1, -1, -1, -1, -1, 0],
    
    # Restrição de estoque máximo às 3pm: (300 - x1) + ... + 900 - x6 ≤ 1000
    [-1, -1, -1, -1, -1, -1],
    
    # Restrição de não deixar estoque overnight: todo chemical X deve ser enviado
    [1, 1, 1, 1, 1, 1],
    
    # Restrições de capacidade máxima de envio (não pode enviar mais do que o estoque disponível)
    [1, 0, 0, 0, 0, 0],
    [1, 1, 0, 0, 0, 0],
    [1, 1, 1, 0, 0, 0],
    [1, 1, 1, 1, 0, 0],
    [1, 1, 1, 1, 1, 0],
    [1, 1, 1, 1, 1, 1]
])

b = np.array([
    540,    # 1000 - (300 + 240) = 460 → -x1 - x2 ≤ 460 → -x1 - x2 ≤ 540 (corrigido)
    140,    # 1000 - (300 + 240 + 600) = -140 → -x1 - x2 - x3 ≤ -140
    -60,    # 1000 - (300 + 240 + 600 + 200) = -340 → -x1 - ... - x4 ≤ -340
    -340,   # 1000 - (300 + 240 + 600 + 200 + 300) = -640 → -x1 - ... - x5 ≤ -640
    -640,   # 1000 - (300 + 240 + 600 + 200 + 300 + 900) = -1540 → -x1 - ... - x6 ≤ -1540
    2540,   # Total produzido: 300+240+600+200+300+900 = 2540
    300,    # x1 ≤ 300
    540,    # x1 + x2 ≤ 540
    1140,   # x1 + x2 + x3 ≤ 1140
    1340,   # x1 + ... + x4 ≤ 1340
    1640,   # x1 + ... + x5 ≤ 1640
    2540    # x1 + ... + x6 ≤ 2540
])

signs = np.array(["<=", "<=", "<=", "<=", "<=", "=", "<=", "<=", "<=", "<=", "<=", "<="])

# Todas as variáveis são não-negativas
nonneg = np.array([True, True, True, True, True, True])

z = 0

x, z = solve_lp("min", A, b, c, z, signs, nonneg)
x, z

(array([ 300.,    0.,  840.,    0.,  500.,  900.,  840., 1280., 1080.,
        1300., 1900.,    0.,  240.,    0.,  200.,    0.,    0.]),
 np.float64(-102400.0))

## 1.2.7


In [40]:
def solve_transportation_hubs_problem(m=3, n=4):
    """
    Solve the transportation problem with hubs.
    
    Parameters:
    m: number of factories
    n: number of stores
    """
    # Criação do problema
    lp = pl.LpProblem("Transportation_With_Hubs", pl.LpMinimize)

    # Variáveis de decisão
    # Transporte das fábricas para os hubs
    factory_to_A = []
    factory_to_B = []
    for i in range(1, m+1):
        factory_to_A.append(pl.LpVariable(f"Factory{i}_to_A", lowBound=0))
        factory_to_B.append(pl.LpVariable(f"Factory{i}_to_B", lowBound=0))
    
    # Transporte dos hubs para as lojas
    A_to_store = []
    B_to_store = []
    for j in range(1, n+1):
        A_to_store.append(pl.LpVariable(f"A_to_Store{j}", lowBound=0))
        B_to_store.append(pl.LpVariable(f"B_to_Store{j}", lowBound=0))

    # Custos de transporte (valores exemplo)
    # Custos das fábricas para os hubs
    cost_to_A = [2, 4, 3]  # a1, a2, a3
    cost_to_B = [3, 1, 2]  # b1, b2, b3
    
    # Custos dos hubs para as lojas
    cost_A_to = [5, 2, 4, 3]  # a'1, a'2, a'3, a'4
    cost_B_to = [3, 6, 2, 1]  # b'1, b'2, b'3, b'4
    
    # Capacidades das fábricas (valores exemplo)
    factory_capacity = [100, 150, 200]  # u1, u2, u3
    
    # Demandas das lojas (valores exemplo)
    store_demand = [80, 120, 90, 110]  # ℓ1, ℓ2, ℓ3, ℓ4

    # Função objetivo (minimizar custo total de transporte)
    total_cost = 0
    for i in range(m):
        total_cost += cost_to_A[i] * factory_to_A[i] + cost_to_B[i] * factory_to_B[i]
    for j in range(n):
        total_cost += cost_A_to[j] * A_to_store[j] + cost_B_to[j] * B_to_store[j]
    
    lp += total_cost, "Total_Transportation_Cost"

    # Restrições de capacidade das fábricas
    for i in range(m):
        lp += factory_to_A[i] + factory_to_B[i] <= factory_capacity[i], f"Factory{i+1}_Capacity"

    # Restrições de demanda das lojas
    for j in range(n):
        lp += A_to_store[j] + B_to_store[j] >= store_demand[j], f"Store{j+1}_Demand"

    # Restrições de conservação nos hubs (entrada = saída)
    total_to_A = pl.lpSum(factory_to_A)
    total_from_A = pl.lpSum(A_to_store)
    lp += total_to_A == total_from_A, "Hub_A_Conservation"
    
    total_to_B = pl.lpSum(factory_to_B)
    total_from_B = pl.lpSum(B_to_store)
    lp += total_to_B == total_from_B, "Hub_B_Conservation"

    # Resolver
    lp.solve(pl.PULP_CBC_CMD(msg=0))

    # Solução
    print("Status:", pl.LpStatus[lp.status])
    
    solution = {}
    print("\nOptimal Transportation Plan:")
    
    print("\nFrom Factories to Hubs:")
    total_cost_factories = 0
    for i in range(m):
        to_A = factory_to_A[i].varValue
        to_B = factory_to_B[i].varValue
        cost_A = to_A * cost_to_A[i]
        cost_B = to_B * cost_to_B[i]
        total_cost_factories += cost_A + cost_B
        solution[f"Factory{i+1}_to_A"] = to_A
        solution[f"Factory{i+1}_to_B"] = to_B
        print(f"  Factory {i+1}: {to_A:.1f} to A (${cost_A:.2f}), {to_B:.1f} to B (${cost_B:.2f})")
    
    print("\nFrom Hubs to Stores:")
    total_cost_stores = 0
    for j in range(n):
        from_A = A_to_store[j].varValue
        from_B = B_to_store[j].varValue
        cost_A = from_A * cost_A_to[j]
        cost_B = from_B * cost_B_to[j]
        total_cost_stores += cost_A + cost_B
        solution[f"A_to_Store{j+1}"] = from_A
        solution[f"B_to_Store{j+1}"] = from_B
        print(f"  Store {j+1}: {from_A:.1f} from A (${cost_A:.2f}), {from_B:.1f} from B (${cost_B:.2f})")
    
    print(f"\nTotal Transportation Cost: ${total_cost_factories + total_cost_stores:,.2f}")
    print(f"  - Factories to Hubs: ${total_cost_factories:,.2f}")
    print(f"  - Hubs to Stores: ${total_cost_stores:,.2f}")
    
    # Verificar conservação
    print(f"\nHub A Conservation: In = {sum(factory_to_A[i].varValue for i in range(m)):.1f}, "
          f"Out = {sum(A_to_store[j].varValue for j in range(n)):.1f}")
    print(f"Hub B Conservation: In = {sum(factory_to_B[i].varValue for i in range(m)):.1f}, "
          f"Out = {sum(B_to_store[j].varValue for j in range(n)):.1f}")
    
    objective_value = pl.value(lp.objective)
    
    return solution, objective_value

solve_transportation_hubs_problem()

Status: Optimal

Optimal Transportation Plan:

From Factories to Hubs:
  Factory 1: 100.0 to A ($200.00), 0.0 to B ($0.00)
  Factory 2: 0.0 to A ($0.00), 150.0 to B ($150.00)
  Factory 3: 20.0 to A ($60.00), 130.0 to B ($260.00)

From Hubs to Stores:
  Store 1: 0.0 from A ($0.00), 80.0 from B ($240.00)
  Store 2: 120.0 from A ($240.00), 0.0 from B ($0.00)
  Store 3: 0.0 from A ($0.00), 90.0 from B ($180.00)
  Store 4: 0.0 from A ($0.00), 110.0 from B ($110.00)

Total Transportation Cost: $1,440.00
  - Factories to Hubs: $670.00
  - Hubs to Stores: $770.00

Hub A Conservation: In = 120.0, Out = 120.0
Hub B Conservation: In = 280.0, Out = 280.0


({'Factory1_to_A': 100.0,
  'Factory1_to_B': 0.0,
  'Factory2_to_A': 0.0,
  'Factory2_to_B': 150.0,
  'Factory3_to_A': 20.0,
  'Factory3_to_B': 130.0,
  'A_to_Store1': 0.0,
  'B_to_Store1': 80.0,
  'A_to_Store2': 120.0,
  'B_to_Store2': 0.0,
  'A_to_Store3': 0.0,
  'B_to_Store3': 90.0,
  'A_to_Store4': 0.0,
  'B_to_Store4': 110.0},
 1440.0)

In [41]:
import numpy as np

# Variáveis: [f1A, f1B, f2A, f2B, f3A, f3B, As1, As2, As3, As4, Bs1, Bs2, Bs3, Bs4]
# Onde: fiA = unidades da fábrica i para hub A, fiB = unidades da fábrica i para hub B
#       Asj = unidades do hub A para loja j, Bsj = unidades do hub B para loja j

m = 3  # número de fábricas
n = 4  # número de lojas

# Custos: [a1, b1, a2, b2, a3, b3, a'1, a'2, a'3, a'4, b'1, b'2, b'3, b'4]
c = np.array([2, 3, 4, 1, 3, 2, 5, 2, 4, 3, 3, 6, 2, 1])  # Custos exemplo

# Matriz A de restrições (9 restrições × 14 variáveis)
A = np.array([
    # Restrições de capacidade das fábricas (f1A + f1B ≤ u1, etc.) - 3 restrições
    [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
    
    # Restrições de demanda das lojas (As1 + Bs1 ≥ ℓ1, etc.) - 4 restrições
    [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1],
    
    # Restrições de conservação nos hubs (entrada = saída) - 2 restrições
    [1, 0, 1, 0, 1, 0, -1, -1, -1, -1, 0, 0, 0, 0],  # Hub A
    [0, 1, 0, 1, 0, 1, 0, 0, 0, 0, -1, -1, -1, -1]   # Hub B
])

# Vetor b (9 elementos - um para cada restrição)
b = np.array([100, 150, 200, 80, 120, 90, 110, 0, 0])  # Removido o elemento extra

signs = np.array(["<=", "<=", "<=", ">=", ">=", ">=", ">=", "=", "="])

# Todas as variáveis são não-negativas
nonneg = np.array([True] * (2*m + 2*n))

z = 0

x, z = solve_lp("min", A, b, c, z, signs, nonneg)
x, z

(array([100.,   0.,   0., 150.,  20., 130.,   0., 120.,   0.,   0.,  80.,
          0.,  90., 110.,   0.,   0.,  50.,   0.,   0.,   0.,   0.]),
 np.float64(-1440.0))

## 1.2.8


In [45]:
def solve_max_margin_classification_fixed():
    """
    Solve the max margin classification problem with fixed, feasible data.
    """
    # Dados fixos e claramente separáveis
    # Healthy (A) - cluster em torno de (2,2)
    A = np.array([
        [1.8, 2.1],
        [2.2, 1.9],
        [1.9, 2.3],
        [2.1, 1.8],
        [2.0, 2.0]
    ])

    # Unhealthy (B) - cluster em torno de (-2,-2)  
    B = np.array([
        [-2.1, -1.9],
        [-1.8, -2.2],
        [-2.2, -1.8],
        [-1.9, -2.1],
        [-2.0, -2.0]
    ])
    
    m, n = A.shape
    p = B.shape[0]
    
    print("Healthy data (A):")
    print(A)
    print("\nUnhealthy data (B):")
    print(B)
    print()
    
    # Criação do problema (MAXIMIZAR margem)
    lp = pl.LpProblem("Max_Margin_Classification", pl.LpMaximize)

    # Variáveis de decisão
    a1 = pl.LpVariable("a1", cat='Continuous')
    a2 = pl.LpVariable("a2", cat='Continuous')
    alpha = pl.LpVariable("alpha", cat='Continuous')
    beta = pl.LpVariable("beta", cat='Continuous')
    gamma = pl.LpVariable("gamma", cat='Continuous')  # Margem = alpha - beta

    # Função objetivo (maximizar a margem)
    lp += gamma, "Maximize_Margin"

    # Restrições para observações healthy: a^T x ≤ α
    for i in range(m):
        constraint_expr = a1 * A[i, 0] + a2 * A[i, 1]
        lp += constraint_expr <= alpha, f"Healthy_Constraint_{i}"

    # Restrições para observações unhealthy: a^T x ≥ β  
    for i in range(p):
        constraint_expr = a1 * B[i, 0] + a2 * B[i, 1]
        lp += constraint_expr >= beta, f"Unhealthy_Constraint_{i}"

    # Restrição que define a margem: γ = α - β
    lp += gamma == alpha - beta, "Margin_Definition"

    # Resolver
    lp.solve(pl.PULP_CBC_CMD(msg=0))

    # Solução
    print("Status:", pl.LpStatus[lp.status])
    
    # Extrair valores das variáveis
    a1_val = a1.varValue
    a2_val = a2.varValue
    alpha_val = alpha.varValue
    beta_val = beta.varValue
    gamma_val = gamma.varValue
    
    a_solution = np.array([a1_val, a2_val])
    
    solution = {
        'a': a_solution,
        'alpha': alpha_val,
        'beta': beta_val,
        'gamma': gamma_val
    }
    
    print(f"\n=== OPTIMAL SOLUTION ===")
    print(f"  a = [{a1_val:.4f}, {a2_val:.4f}]")
    print(f"  α = {alpha_val:.4f}")
    print(f"  β = {beta_val:.4f}")
    print(f"  γ = α - β = {gamma_val:.4f} (margin)")
    
    # Verificar restrições
    print(f"\n=== CONSTRAINT VERIFICATION ===")
    
    # Verificar observações healthy
    print("Healthy observations (should be ≤ α):")
    healthy_ok = True
    for i in range(m):
        value = np.dot(a_solution, A[i])
        status = "✓" if value <= alpha_val + 1e-6 else "✗"
        print(f"  {status} A[{i}]: a^T x = {value:.4f} ≤ α = {alpha_val:.4f}")
        if value > alpha_val + 1e-6:
            healthy_ok = False
    
    # Verificar observações unhealthy
    print("\nUnhealthy observations (should be ≥ β):")
    unhealthy_ok = True
    for i in range(p):
        value = np.dot(a_solution, B[i])
        status = "✓" if value >= beta_val - 1e-6 else "✗"
        print(f"  {status} B[{i}]: a^T x = {value:.4f} ≥ β = {beta_val:.4f}")
        if value < beta_val - 1e-6:
            unhealthy_ok = False
    
    if healthy_ok and unhealthy_ok:
        print(f"\n✓ All constraints satisfied!")
    else:
        print(f"\n✗ Some constraints violated!")
    
    # Mostrar hiperplanos
    print(f"\n=== DECISION BOUNDARIES ===")
    print(f"  Healthy region: a^T x ≤ {alpha_val:.4f}")
    print(f"  Unhealthy region: a^T x ≥ {beta_val:.4f}")
    print(f"  Margin width: {gamma_val:.4f}")
    
    # Classificar alguns pontos de teste
    print(f"\n=== TEST POINT CLASSIFICATION ===")
    test_points = [
        [3.0, 3.0],   # Deve ser healthy
        [-3.0, -3.0], # Deve ser unhealthy  
        [0.0, 0.0]    # Na região de margem
    ]
    
    for point in test_points:
        value = np.dot(a_solution, point)
        if value <= alpha_val:
            classification = "HEALTHY"
        elif value >= beta_val:
            classification = "UNHEALTHY"
        else:
            classification = "IN MARGIN"
        print(f"  Point {point}: a^T x = {value:.4f} → {classification}")
    
    objective_value = pl.value(lp.objective)
    
    return solution, objective_value

solve_max_margin_classification_fixed()

Healthy data (A):
[[1.8 2.1]
 [2.2 1.9]
 [1.9 2.3]
 [2.1 1.8]
 [2.  2. ]]

Unhealthy data (B):
[[-2.1 -1.9]
 [-1.8 -2.2]
 [-2.2 -1.8]
 [-1.9 -2.1]
 [-2.  -2. ]]

Status: Unbounded

=== OPTIMAL SOLUTION ===
  a = [0.0000, 0.0000]
  α = 0.0000
  β = 0.0000
  γ = α - β = 0.0000 (margin)

=== CONSTRAINT VERIFICATION ===
Healthy observations (should be ≤ α):
  ✓ A[0]: a^T x = 0.0000 ≤ α = 0.0000
  ✓ A[1]: a^T x = 0.0000 ≤ α = 0.0000
  ✓ A[2]: a^T x = 0.0000 ≤ α = 0.0000
  ✓ A[3]: a^T x = 0.0000 ≤ α = 0.0000
  ✓ A[4]: a^T x = 0.0000 ≤ α = 0.0000

Unhealthy observations (should be ≥ β):
  ✓ B[0]: a^T x = 0.0000 ≥ β = 0.0000
  ✓ B[1]: a^T x = 0.0000 ≥ β = 0.0000
  ✓ B[2]: a^T x = 0.0000 ≥ β = 0.0000
  ✓ B[3]: a^T x = 0.0000 ≥ β = 0.0000
  ✓ B[4]: a^T x = 0.0000 ≥ β = 0.0000

✓ All constraints satisfied!

=== DECISION BOUNDARIES ===
  Healthy region: a^T x ≤ 0.0000
  Unhealthy region: a^T x ≥ 0.0000
  Margin width: 0.0000

=== TEST POINT CLASSIFICATION ===
  Point [3.0, 3.0]: a^T x = 0.0000 → H

({'a': array([0., 0.]), 'alpha': 0.0, 'beta': 0.0, 'gamma': 0.0}, 0.0)

In [46]:
import numpy as np

# Dados fixos e factíveis
# Healthy (A) - cluster em torno de (2,2)
A = np.array([
    [1.8, 2.1],
    [2.2, 1.9],
    [1.9, 2.3],
    [2.1, 1.8],
    [2.0, 2.0]
])

# Unhealthy (B) - cluster em torno de (-2,-2)  
B = np.array([
    [-2.1, -1.9],
    [-1.8, -2.2],
    [-2.2, -1.8],
    [-1.9, -2.1],
    [-2.0, -2.0]
])

m, n = A.shape
p, n2 = B.shape

# Variáveis: [a1, a2, α, β, γ] onde γ = α - β (margem a ser maximizada)
c = np.zeros(n + 3)
c[-1] = -1  # Maximizar γ = α - β (minimizar -γ)

# Matriz de restrições
total_constraints = m + p + 1  # m para A + p para B + 1 para γ = α - β
A_lp = np.zeros((total_constraints, n + 3))

# Restrições para A (healthy): a^T x_i ≤ α
for i in range(m):
    A_lp[i, :n] = A[i]      # Coeficientes de a
    A_lp[i, n] = -1         # Coeficiente de -α
    A_lp[i, n+1] = 0        # Coeficiente de β
    A_lp[i, n+2] = 0        # Coeficiente de γ

# Restrições para B (unhealthy): a^T x_j ≥ β → -a^T x_j + β ≤ 0
for j in range(p):
    A_lp[m + j, :n] = -B[j]  # Coeficientes de -a
    A_lp[m + j, n] = 0       # Coeficiente de α
    A_lp[m + j, n+1] = 1     # Coeficiente de β  
    A_lp[m + j, n+2] = 0     # Coeficiente de γ

# Restrição adicional: γ = α - β → α - β - γ = 0
A_lp[-1, n] = 1      # α
A_lp[-1, n+1] = -1   # -β
A_lp[-1, n+2] = -1   # -γ

b = np.zeros(total_constraints)  # Todas as desigualdades são ≤ 0, mais a igualdade

signs = np.array(["<="] * (m + p) + ["="])

# Variáveis livres (sem restrição de não-negatividade)
nonneg = np.array([False] * (n + 3))

z = 0

x, z = solve_lp("min", A_lp, b, c, z, signs, nonneg)
x, z

The problem is unbounded.


(None, None)